In [121]:
import numpy as np
from scipy.ndimage import distance_transform_edt
import torch

def calculate_score_faster(mask_gt, mask_pred, r):
    
    mask_gt = mask_gt.cpu().detach().numpy().squeeze()
    mask_pred = mask_pred.cpu().detach().numpy().squeeze()
    
    height, width = mask_gt.shape
    scores = np.zeros((height, width))  


    class_distances = {}
    for class_value in np.unique(mask_gt):
        class_mask = (mask_gt == class_value)
        class_distances[class_value] = distance_transform_edt(~class_mask)


    for y in range(height):
        for x in range(width):
            
            gt_class = mask_gt[y, x]
            pred_class = mask_pred[y, x]


            dist_map = class_distances[gt_class]


            near_match = dist_map[y, x] <= r


            exact_match = pred_class == gt_class
            tolerance_match = abs(pred_class - gt_class) <= 1


            pixel_score = 0
            if exact_match and near_match:
                pixel_score = max(pixel_score, 50)
            if exact_match:
                pixel_score = max(pixel_score, 100)
            if tolerance_match and not exact_match:
                pixel_score = max(pixel_score, 50)
            if tolerance_match and near_match and not exact_match:
                pixel_score = max(pixel_score, 25)


            scores[y, x] = pixel_score


    max_score = height * width * 100
    normalized_score = np.sum(scores) / max_score

    return normalized_score

def calculate_score_faster_isolation_beckground_excluded(mask_gt, mask_pred, r, penalty_points):
    height, width = mask_gt.shape
    scores = np.zeros((height, width))  # Initialize score array

    # Precompute distance maps for each class present in ground truth
    class_distances = {}
    for class_value in np.unique(mask_gt):
        if class_value == 0:
            continue  # Skip background
        class_mask = (mask_gt == class_value)
        class_distances[class_value] = distance_transform_edt(~class_mask)

    # Create a convolution kernel to count neighbors
    kernel = np.array([[1, 1, 1],
                       [1, 0, 1],
                       [1, 1, 1]])

    # Iterate over each pixel
    for y in range(height):
        for x in range(width):
            pred_class = mask_pred[y, x]
            if pred_class == 0:
                continue  # Skip background in predictions

            gt_class = mask_gt[y, x]

            # Retrieve the precomputed distance map for the current ground truth class
            if gt_class in class_distances:
                dist_map = class_distances[gt_class]
            else:
                dist_map = np.full((height, width), np.inf)  # No valid distance map for this class

            # Determine if the predicted class is near the ground truth class
            near_match = dist_map[y, x] <= r

            # Determine the exact and tolerance matches
            exact_match = pred_class == gt_class
            tolerance_match = abs(pred_class - gt_class) <= 1

            # Isolation check with 1-level tolerance
            neighbors_pred = mask_pred[max(y-1, 0):min(y+2, height), max(x-1, 0):min(x+2, width)]
            isolation_pred = np.sum(np.abs(neighbors_pred - pred_class) <= 1) <= 2

            neighbors_gt = mask_gt[max(y-1, 0):min(y+2, height), max(x-1, 0):min(x+2, width)]
            isolation_gt = np.sum(np.abs(neighbors_gt - gt_class) <= 1) <= 2

            # Compute the maximum score for the pixel based on the conditions
            pixel_score = 0
            if exact_match:
                pixel_score = 100
            elif near_match and exact_match:
                pixel_score = max(pixel_score, 50)
            elif exact_match and tolerance_match:
                pixel_score = max(pixel_score, 50)
            elif near_match and tolerance_match:
                pixel_score = max(pixel_score, 25)

            # Apply penalty for isolated prediction pixels
            if isolation_pred and not isolation_gt:
                pixel_score -= penalty_points

            # Assign the computed score to the pixel
            scores[y, x] = pixel_score
    # Normalize the total score by the maximum possible score
    non_zero_mask = mask_pred > 0
    max_score = np.sum(non_zero_mask) * 100  # Maximum possible score for non-zero pixels
    normalized_score = np.sum(scores) / max_score if max_score > 0 else 0

    return normalized_score
            
def calculate_score_faster_isolation(mask_gt, mask_pred, r, penalty_points):
    height, width = mask_gt.shape
    scores = np.zeros((height, width))  # Initialize score array

    # Precompute distance maps for each class present in ground truth
    class_distances = {}
    for class_value in np.unique(mask_gt):
        class_mask = (mask_gt == class_value)
        class_distances[class_value] = distance_transform_edt(~class_mask)

    # Create a convolution kernel to count neighbors
    kernel = np.array([[1, 1, 1],
                       [1, 0, 1],
                       [1, 1, 1]])

    # Iterate over each pixel
    for y in range(height):
        for x in range(width):
            gt_class = mask_gt[y, x]
            pred_class = mask_pred[y, x]

            # Retrieve the precomputed distance map for the current ground truth class
            dist_map = class_distances[gt_class]

            # Determine if the predicted class is near the ground truth class
            near_match = dist_map[y, x] <= r

            # Determine the exact and tolerance matches
            exact_match = pred_class == gt_class
            tolerance_match = abs(pred_class - gt_class) <= 1

            # Isolation check with 1-level tolerance
            neighbors_pred = mask_pred[max(x-1, 0):min(x+2, width), max(y-1, 0):min(y+2, height)]
            isolation_pred = np.sum(np.abs(neighbors_pred - pred_class) <= 1) <= 2
#             print(x,y,isolation_pred)

            neighbors_gt = mask_gt[max(x-1, 0):min(x+2, width), max(y-1, 0):min(y+2, height)]
            isolation_gt = np.sum(np.abs(neighbors_gt - gt_class) <= 1) <= 2
#             print(x,y,isolation_gt)

            # Compute the maximum score for the pixel based on the conditions
            pixel_score = 0
            if exact_match:
                pixel_score = 100
            elif near_match and exact_match:
                pixel_score = max(pixel_score, 50)
            elif exact_match and tolerance_match:
                pixel_score = max(pixel_score, 50)
            elif near_match and tolerance_match:
                pixel_score = max(pixel_score, 25)

            # Apply penalty for isolated prediction pixels
            if isolation_pred and not isolation_gt:
                pixel_score -= penalty_points

            # Assign the computed score to the pixel
            scores[y, x] = pixel_score

    # Normalize the total score by the maximum possible score
    max_score = height * width * 100
    normalized_score = np.sum(scores) / max_score

    return normalized_score

def calculate_score(mask_gt, mask_pred, r):
    height, width = mask_gt.shape
    scores = np.zeros((height, width))  # Initialize score array

    # Iterate over each pixel
    for y in range(height):
        for x in range(width):
            gt_class = mask_gt[y, x]
            pred_class = mask_pred[y, x]

            # Distance transform for the current ground truth class
            class_mask = (mask_gt == gt_class)
            class_distances = distance_transform_edt(~class_mask)

            # Conditions for scoring
            exact_match = pred_class == gt_class
            tolerance_match = abs(pred_class - gt_class) <= 1
            near_match = class_distances[y, x] <= r

            # Assign scores based on the conditions
            current_score = 0
            if exact_match:
                current_score = 100
            if near_match and exact_match:
                current_score = max(current_score, 50)
            if exact_match and tolerance_match:
                current_score = max(current_score, 50)
            if near_match and tolerance_match:
                current_score = max(current_score, 25)

            # Store the maximum score achieved for this pixel
            scores[y, x] = current_score

    # Normalize the total score by the maximum possible score
    max_score = height * width * 100
    normalized_score = np.sum(scores) / max_score

    return normalized_score

def calculate_new_score(mask_gt, mask_pred, r):
    height, width = mask_gt.shape
    total_score = 0

    # Initialize score and distance arrays
    scores = np.zeros((height, width))

    # Iterate over each unique class in the ground truth
    for class_value in np.unique(mask_gt):
        # Create a mask for the current class
        class_mask = (mask_gt == class_value)
        print("class_mask")
        print(class_mask)
        
        # Compute the distance transform for the current class
        class_distances = distance_transform_edt(~class_mask)
        print("class_distances")
        print(class_distances)

        # Evaluate predictions within radius r for this class
        near_mask = (class_distances <= r)
        print("near_mask")
        print(near_mask)
        exact_mask = (mask_pred == class_value)
        print("exact_mask")
        print(exact_mask)

        # Scoring: exact location and same class
        scores += 100 * (class_mask & exact_mask)
        
        # Scoring: within r distance and same class
        scores += 50 * near_mask & exact_mask
        
        # Scoring: exact location but class mismatch within one level tolerance
        tolerance_mask = (np.abs(mask_pred - class_value) == 1)
        scores += 50 * (class_mask & tolerance_mask)

        # Scoring: within r distance and class mismatch within one level tolerance
        scores += 25 * (near_mask & tolerance_mask)

    # Normalize the total score by the maximum possible score
    max_score = height * width * 100
    normalized_score = np.sum(scores) / max_score

    return normalized_score

def mIOU(label, pred, num_classes=9):
#     pred = F.softmax(pred, dim=1)              
#     pred = torch.argmax(pred, dim=1).squeeze(1)
    iou_list = list()
    present_iou_list = list()

#     pred = pred.view(-1)
#     label = label.view(-1)
    # Note: Following for loop goes from 0 to (num_classes-1)
    # and ignore_index is num_classes, thus ignore_index is
    # not considered in computation of IoU.
    for sem_class in range(num_classes):
        pred_inds = (pred == sem_class)
        target_inds = (label == sem_class)
        if target_inds.long().sum().item() == 0:
            iou_now = float('nan')
        else: 
            intersection_now = (pred_inds[target_inds]).long().sum().item()
            union_now = pred_inds.long().sum().item() + target_inds.long().sum().item() - intersection_now
            iou_now = float(intersection_now) / float(union_now)
            present_iou_list.append(iou_now)
            # print(iou_now)
        iou_list.append(iou_now)
    return iou_list, present_iou_list, np.mean(present_iou_list)

# # Example usage
# mask_gt = np.array([[1, 2, 3], [1, 2, 3], [1, 2, 3]])
# mask_pred = np.array([[2, 2, 3], [1, 2, 3], [1, 2, 3]])
# r = 1

# print(f"Normalized Score: {calculate_score(mask_gt, mask_pred, r):.2f}")

In [120]:
gt =np.array(([0,0,2,0,0],

              [0,0,0,2,0],

              [0,0,0,0,2],

              [0,0,0,2,0],

              [0,0,2,0,0]))

b = np.array(([0,1,0,0,0],

              [0,0,1,0,0],

              [0,0,0,1,0],

              [0,0,1,0,0],

              [0,1,0,0,0]))

c = np.array(([0,0,2,0,0],

              [0,0,0,2,0],

              [0,0,0,0,2],

              [0,0,0,2,0],

              [0,0,2,0,0]))

d = np.array(([0,0,1,0,0],

              [0,0,0,1,0],

              [0,0,0,0,1],

              [0,0,0,1,0],

              [0,0,1,0,0]))

r = 1

pred = b

iou_list, present_iou_list, iou_score = mIOU(torch.tensor(gt), torch.tensor(pred))

# print(f"Normalized Score: {calculate_score_faster_isolation_beckground_excluded(gt, pred, r, penalty_points=10):.2f}")
print(f"Normalized New Score: {calculate_score(gt, pred, r):.2f}")
print(f"mIoU Score: {iou_score}")

Normalized New Score: 0.65
mIoU Score: 0.3


In [90]:
a = calculate_score_faster_isolation_beckground_excluded(gt, pred, r, penalty_points=100)

In [91]:
a